# 🌍 Phase 1 – Python Geoscience Foundations

## ✨ Objective
In this notebook, we explore core libraries for geospatial programming in Python.

---

## 1. Environment Overview

In [ ]:
import geopandas as gpd
import rasterio
import fiona
import shapely
import pyproj
import matplotlib.pyplot as plt

## 2. Reading Vector Data with Fiona / GeoPandas
🧪 Task: Load and inspect a shapefile

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.head()

## 3. Projections with PyProj
🧪 Task: Transform CRS of a dataset

In [ ]:
world = world.to_crs("EPSG:3857")
world.plot()

## 4. Raster Basics with Rasterio
🧪 Task: Read and inspect raster metadata

In [ ]:
# Note: You'll need to replace "path_to_raster.tif" with an actual file path
# try:
#     with rasterio.open("path_to_raster.tif") as src:
#         print(src.profile)
#         data = src.read(1)
# except rasterio.RasterioIOError as e:
#     print(f"Error opening raster file: {e}")
#     print("Please provide a valid path to a raster file.")

## 5. Mini Project: Reproject and Plot Raster + Vector Data
Combine vector and raster layers with different CRSes and visualize them on a common map.

In [ ]:
# Placeholder for the mini-project code

## 📝 Summary and Reflections
*   What we covered
*   Limitations or performance issues
*   Ideas for automation or future notebooks